In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split


from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import folium
from folium.plugins import HeatMap
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import lightgbm as lgb

from sklearn.metrics import mean_squared_error

from pandas import DataFrame

import matplotlib.pyplot as plt

# #evaluation = pd.DataFrame({'Model': [],
#                            'Details':[],
#                            'Root Mean Squared Error (RMSE)':[],
#                            'R-squared (training)':[],
#                            'Adjusted R-squared (training)':[],
#                            'R-squared (test)':[],
#                            'Adjusted R-squared (test)':[],
#                            '5-Fold Cross Validation':[]})
                           
all_datadf=pd.read_csv('good_dataset.csv')




/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
#remove outliers function
def remove_outliers(df,columns,n_std):
    for col in columns:
        #print('Working on column: {}'.format(col))
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
    print(str(round(mean,2)) + ', ' +  str(round(sd,2)))
    #print(df[col] <= mean+(n_std*sd))    
    return df

In [3]:
#basic cleaning

all_datadf=all_datadf.fillna(0)
all_datadf=all_datadf.iloc[:, 2:]
all_datadf=all_datadf.drop_duplicates()

#price cleaning, drop 0s
all_datadf['price']=pd.to_numeric(all_datadf['price'], errors='coerce')
all_datadf=all_datadf[all_datadf['price'] > 0]


#numbedrooms cleaning
all_datadf=all_datadf[all_datadf['numberOfBedrooms'] < 10]
all_datadf=all_datadf[all_datadf['surfaceAreaOfPlot'] < 15000]

all_datadf.describe()


,Unnamed: 0,immowebId,postalCode,price,numberOfBedrooms,hasBathroom,hasLivingRoom,livableArea,hasKitchen,fullyEquipedKitchen,...,hasGarden,isFurnished,hasSwimmingPool,hasRenewableEnergy,surfaceAreaOfPlot,surfaceOfBuildableLand,numberOfFacades,gardenArea,surfaceOfLand,energyInfo
count,1.156900e+04,1.156900e+04,11569.000000,1.156900e+04,11569.000000,11569.000000,11569.000000,11569.000000,11569.000000,11569.000000,...,11569.000000,11569.000000,11569.000000,11569.000000,11569.000000,11569.000000,11569.000000,11569.000000,11569.000000,11569.0
mean,1.000312e+07,1.000312e+07,5255.386550,3.903960e+05,2.776039,1.091451,0.389057,133.367534,0.675080,0.604547,...,0.322500,0.021177,0.017460,0.044602,428.848993,21.262771,1.945371,108.828853,22.797908,0.0
std,1.173528e+05,1.173528e+05,3023.807493,3.802809e+05,1.311612,0.838396,0.487557,130.011830,0.468365,0.488969,...,0.467453,0.143981,0.130985,0.206437,1139.490268,125.968238,1.488066,1169.723974,51.746654,0.0
min,9.699111e+06,9.699111e+06,1000.000000,5.170000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,9.981536e+06,9.981536e+06,2300.000000,2.250000e+05,2.000000,1.000000,0.000000,75.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,1.005934e+07,1.005934e+07,4920.000000,2.990000e+05,3.000000,1.000000,0.000000,110.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.0
75%,1.008043e+07,1.008043e+07,8420.000000,4.250000e+05,3.000000,1.000000,1.000000,172.000000,1.000000,1.000000,...,1.000000,0.000000,0.000000,0.000000,402.000000,0.000000,3.000000,0.000000,0.000000,0.0
max,1.009538e+07,1.009538e+07,9992.000000,6.950000e+06,9.000000,12.000000,1.000000,4940.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,14884.000000,6546.000000,8.000000,60000.000000,2022.000000,0.0


In [4]:
#epc, state and property type to continuous
#energy type fill na

epc_dict={'A++': 9, 'A+': 8, 'A' : 7, 'B_A+': 6,'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1, 'G_D' : 1, 'G_B' : 1, 'E_D': 2}

state_dict={'TO_RESTORE' : 1, 'TO_RENOVATE' : 2, 'GOOD' : 3, 'TO_BE_DONE_UP': '4', 'JUST_RENOVATED' : 5, 'AS_NEW' : 6}

property_type_dict={'HOUSE' : 1, 'APARTMENT': 0}

energy_type_dict={'0': 'not_specified'}

all_datadf=all_datadf.replace(epc_dict)
all_datadf=all_datadf.replace(state_dict)
all_datadf=all_datadf.replace(property_type_dict)
all_datadf['energyHeatingType']=all_datadf['energyHeatingType'].replace(energy_type_dict)

all_datadf['epcScore']=pd.to_numeric(all_datadf['epcScore'], errors='coerce')
all_datadf['stateOfBuilding']=pd.to_numeric(all_datadf['stateOfBuilding'], errors='coerce')


In [5]:
#postal codes in blocks til 50

all_datadf['postalCode']=all_datadf['postalCode'] // 25 * 25
postalCodes=all_datadf['postalCode'].unique()
#postalCodesdf=pd.DataFrame(postalCodes, columns=['postalCodes'])
all_datadf['postalCode']=all_datadf['postalCode'].astype('string')



In [6]:
from statistics import median

#most interesting dataframe
bestdf=all_datadf[['price', 'stateOfBuilding', 'epcScore', 'postalCode', 'province', 'typeOfProperty', 'numberOfBedrooms', 'livableArea', 'surfaceAreaOfPlot']]




In [8]:
#removing outliers, if used with besthousedf brings down performance to 0.53

# outlier_columns=all_datadf[['price', 'livableArea']]

# bestdf=remove_outliers(bestdf, outlier_columns, 1)

# bestdf.describe()

94.83, 58.5


,price,stateOfBuilding,epcScore,typeOfProperty,numberOfBedrooms,livableArea,surfaceAreaOfPlot
count,6988.000000,6988.000000,6988.000000,6988.000000,6988.000000,6988.000000,6988.000000
mean,253869.258729,2.560819,2.601317,0.395249,2.248140,77.978964,168.608615
std,85562.704699,2.352296,2.540336,0.488939,0.930667,47.359313,497.823875
min,517.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,195000.000000,0.000000,0.000000,0.000000,2.000000,53.000000,0.000000
50%,255000.000000,3.000000,2.000000,0.000000,2.000000,89.000000,0.000000
75%,312579.500000,5.000000,5.000000,1.000000,3.000000,111.000000,155.000000
max,430500.000000,6.000000,9.000000,1.000000,9.000000,153.000000,12075.000000


In [7]:
#check 0 count
num0s = bestdf.groupby('typeOfProperty').agg(lambda x: x.eq(0).sum())

#median and means per house and appt category
mediandf=bestdf.groupby('typeOfProperty').agg(lambda x: x.median())
meandf=bestdf.groupby('typeOfProperty').agg(lambda x: x.mean())

#medians from state and epc
medianstatehouse=mediandf.iloc[1][2]
medianstateappt=mediandf.iloc[0][2]
medianepchouse=mediandf.iloc[1][3]
medianepcappt=mediandf.iloc[0][3]

#group houses and appts to see distribution of data
besthousedf=bestdf[bestdf['typeOfProperty'] == 1]

bestapptdf=bestdf[bestdf['typeOfProperty'] == 0]
#bestapptdf=bestapptdf.drop(['surfaceAreaOfPlot'], axis=1)

descriptionhouse=besthousedf.describe()
descriptionappt=bestapptdf.describe()



In [8]:
#fill in state and epcs with medians, with state med worse score
#also without epc and state entirely gives worse score
#with filled state and epc medians gives better score
besthousedf['epcScore']=besthousedf['epcScore'].apply(lambda x: medianepchouse if x == 1 else medianepcappt)
besthousedf['stateOfBuilding']=besthousedf['stateOfBuilding'].apply(lambda x: medianstatehouse if x == 1 else medianstateappt)

#
bestapptdf['epcScore']=bestapptdf['epcScore'].apply(lambda x: medianepcappt if x == 0 else x)
bestapptdf['stateOfBuilding']=bestapptdf['stateOfBuilding'].apply(lambda x: medianstatehouse if x == 0 else x)

bestdf.describe()
besthousedf.describe()
#bestapptdf.describe()


,price,stateOfBuilding,epcScore,typeOfProperty,numberOfBedrooms,livableArea,surfaceAreaOfPlot
count,6.341000e+03,6341.000000,6341.000000,6341.0,6341.000000,6341.000000,6341.000000
mean,4.197463e+05,2.006150,2.085475,1.0,3.398360,166.156758,782.424539
std,3.989356e+05,0.078189,0.279610,0.0,1.316586,159.537883,1446.529522
min,1.462000e+03,2.000000,2.000000,1.0,0.000000,0.000000,0.000000
25%,2.300000e+05,2.000000,2.000000,1.0,3.000000,89.000000,145.000000
50%,3.340000e+05,2.000000,2.000000,1.0,3.000000,154.000000,350.000000
75%,4.650000e+05,2.000000,2.000000,1.0,4.000000,219.000000,807.000000
max,6.400000e+06,3.000000,3.000000,1.0,9.000000,4940.000000,14884.000000


In [9]:
#here choose for house or appt, besthousedf or bestapptdf


y_col='price'
y=besthousedf[y_col]
X=(besthousedf[bestapptdf.columns.drop(y_col)]).reset_index(drop=True)

In [10]:

enc=OneHotEncoder(handle_unknown='ignore', sparse=False)

prov_encodeddf=pd.DataFrame(enc.fit_transform(X[['province']]), columns=enc.get_feature_names()).reset_index(drop=True)
postal_encodeddf=pd.DataFrame(enc.fit_transform(X[['postalCode']]), columns=enc.get_feature_names()).reset_index(drop=True)

encodedX=X[['stateOfBuilding', 'numberOfBedrooms', 'surfaceAreaOfPlot', 'livableArea']].join(prov_encodeddf)
encodedX=encodedX.join(postal_encodeddf)
X=encodedX



In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:

from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()

X_train[['surfaceAreaOfPlot', 'livableArea']]=scaler.fit_transform(X_train[['surfaceAreaOfPlot', 'livableArea']])
X_test[['surfaceAreaOfPlot', 'livableArea']]=scaler.fit_transform(X_test[['surfaceAreaOfPlot', 'livableArea']])




In [13]:
rfc = RandomForestRegressor()
parameters = {
    "n_estimators":[5,10,50,100,250],
    "max_depth":[2,4,8,16,32,50,None]
    
}

from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(rfc,parameters,cv=5)
cv.fit(X_train,y_train.values.ravel())

def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')

display(cv)

Best parameters are: {'max_depth': 32, 'n_estimators': 50}


0.334 + or -0.067 for the {'max_depth': 2, 'n_estimators': 5}
0.345 + or -0.062 for the {'max_depth': 2, 'n_estimators': 10}
0.352 + or -0.062 for the {'max_depth': 2, 'n_estimators': 50}
0.355 + or -0.065 for the {'max_depth': 2, 'n_estimators': 100}
0.353 + or -0.064 for the {'max_depth': 2, 'n_estimators': 250}
0.399 + or -0.075 for the {'max_depth': 4, 'n_estimators': 5}
0.439 + or -0.07 for the {'max_depth': 4, 'n_estimators': 10}
0.447 + or -0.066 for the {'max_depth': 4, 'n_estimators': 50}
0.444 + or -0.065 for the {'max_depth': 4, 'n_estimators': 100}
0.45 + or -0.061 for the {'max_depth': 4, 'n_estimators': 250}
0.52 + or -0.057 for the {'max_depth': 8, 'n_estimators': 5}
0.526 + or -0.055 for the {'max_depth': 8, 'n_estimators': 10}
0.541 + or -0.053 for the {'max_depth': 8, 'n_estimators': 50}
0.54 + or -0.047 for the {'max_depth': 8, 'n_estimators': 100}
0.545 + or -0.043 for the {'max_depth': 8, 'n_estimators': 

In [16]:
model = RandomForestRegressor(n_estimators = 300, max_depth=40, random_state=42)
model.fit(X_train,y_train)

RandomForestRegressor(max_depth=40, n_estimators=300, random_state=42)

In [17]:
from math import sqrt

y_pred = model.predict(X_test)
print(metrics.r2_score(y_test,y_pred))
print(metrics.mean_absolute_error(y_test,y_pred))
print(sqrt(metrics.mean_squared_error(y_test,y_pred)))

0.6860519276682056
115473.61698434602
